# Assigment 3

In [1]:
import numpy as np
import pandas as pd
import sklearn
import string
from tqdm import tqdm_notebook as tqdm

In [2]:
path_to_data = "../data/"

# paht to imbd
imbd_train = "IMDB-train.txt"
imbd_val = "IMDB-valid.txt"
imbd_test = "IMDB-test.txt"

# paht to yelp
yelp_train = "yelp-train.txt"
yelp_val = "yelp-valid.txt"
yelp_test = "yelp-test.txt"

In [3]:
# get the data as df
df_imbd_train = pd.read_csv(path_to_data + yelp_train, sep="\t",
                            header=None).rename(columns={0: "review", 1: "class"})
df_imbd_test = pd.read_csv(path_to_data + yelp_test, sep="\t",
                            header=None).rename(columns={0: "review", 1: "class"})
df_imbd_val = pd.read_csv(path_to_data + yelp_val, sep="\t",
                            header=None).rename(columns={0: "review", 1: "class"})

df_yelp_train = pd.read_csv(path_to_data + yelp_train, sep="\t",
                            header=None).rename(columns={0: "review", 1: "class"})
df_yelp_test = pd.read_csv(path_to_data + yelp_test, sep="\t",
                            header=None).rename(columns={0: "review", 1: "class"})
df_yelp_val = pd.read_csv(path_to_data + yelp_val, sep="\t",
                            header=None).rename(columns={0: "review", 1: "class"})

## Part 1

In [4]:
# count frequency of normalized x
def normalize_df(e):
    translator = str.maketrans('', '', string.punctuation)
    normalizer = lambda x: x.lower().translate(translator) # lower and remove punctuation
    return list(filter(None, ' '.join(list(map(normalizer, np.array(e.review)))).split(' ')))

def normalize_str(e):
    translator = str.maketrans('', '', string.punctuation)
    normalizer = lambda x: x.lower().translate(translator) # lower and remove punctuation
    return list(filter(None, list(map(normalizer, e.split(' ')))))
    
    
def count_word_frequency(words):
    occurence = {}
    for w in words:
        if w in occurence:
            occurence[w] += 1
        else:
            occurence[w] = 1
    return occurence

def get_most_n_frequent(data, n):
    normed = normalize_df(data)
    count_dict = count_word_frequency(normed)
    n_sorted_words = (sorted(count_dict.items(), key=lambda kv: kv[1], reverse = True)[:n])
    return [k for k,v in n_sorted_words]
    
def gen_BBoW(data, data_train = None, n = 10000, bin = True):
    most_freq = []
    df_vector = pd.DataFrame(columns=['review', 'class'])

    # check if we use the data to generate the train or not
    if data_train is None:
        most_freq = get_most_n_frequent(data, n)
    else:
        most_freq = get_most_n_frequent(data_train, n)
    
    for review, class_id in tqdm(zip(data.review, data['class']), total = len(data.review)):
        vector = np.zeros(n)        
        for word in normalize_str(review):
            if word in most_freq:
                i = most_freq.index(word)
                if bin:
                    vector[i] = 1
                else:
                    vector[i] += 1
        sum_ = sum(vector)
        if not bin:
            vector = [float(e/sum_) for e in vector]
        df_vector = df_vector.append({'review': vector, 'class': class_id}, ignore_index=True)
    return df_vector

In [ ]:
# binary bag-of-words representation
df_imbd_train_BBoW = gen_BBoW(df_imbd_train, df_imbd_train, bin = True)
df_yelp_train_BBoW = gen_BBoW(df_yelp_train, df_yelp_train, bin = True)
df_imbd_test_BBoW = gen_BBoW(df_imbd_test, df_imbd_train, bin = True)
df_yelp_test_BBoW = gen_BBoW(df_yelp_test, df_yelp_train, bin = True)
df_imbd_val_BBoW = gen_BBoW(df_imbd_val, df_imbd_train, bin = True)
df_yelp_val_BBoW = gen_BBoW(df_yelp_val, df_yelp_train, bin = True)

# frequency bag-of-words representation
df_imbd_train_FBoW = gen_BBoW(df_imbd_train, df_imbd_train, bin = False)
df_yelp_train_FBoW = gen_BBoW(df_yelp_train, df_yelp_train, bin = False)
df_imbd_test_FBoW = gen_BBoW(df_imbd_test, df_imbd_train, bin = False)
df_yelp_test_FBoW = gen_BBoW(df_yelp_test, df_yelp_train, bin = False)
df_imbd_val_FBoW = gen_BBoW(df_imbd_val, df_imbd_train, bin = False)
df_yelp_val_FBoW = gen_BBoW(df_yelp_val, df_yelp_train, bin = False)

HBox(children=(IntProgress(value=0, max=7000), HTML(value='')))

In [ ]:
print(df_yelp_test_FBoW)